# Example: Dual Solution of the Apple versus Orange Problem
This example will familiarize students with the [linear programming dual problem](https://en.wikipedia.org/wiki/Linear_programming). We'll compute the optimal solution to a simple constrained optimization problem using Julia's optimization ecosystem, including the VLDataScienceMachineLearningPackage and JuMP with the GLPK solver.


> __Learning objectives__
> 
> At the end of this example, students will be able to:
> * __Problem Formulation and Setup:__ Students will learn to formulate a constrained optimization problem as a linear program by defining decision variables, objective functions with marginal utilities, and budget constraints. They will understand how to set appropriate variable bounds based on economic reasoning and translate real-world constraints into mathematical form.
> * __Computational Solution Methods:__ Students will implement and solve linear programming problems using Julia's optimization ecosystem, including the VLDataScienceMachineLearningPackage and JuMP with the GLPK solver. They will learn to interpret solution dictionaries, handle solver errors with try-catch blocks, and extract optimal values and objective function results.
> * __Economic Analysis and Interpretation:__ Students will analyze corner solutions in linear utility maximization problems and understand the three fundamental cases that arise from comparing budget line slopes to indifference curve slopes. They will learn to predict solution behavior based on marginal utility coefficients and recognize when multiple optimal solutions exist in the special case of equal marginal utility ratios.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Constants and Problem Data
Let's set the prices, $\alpha_{i}$, and budget for our `Apple` versus `Orange` problem. We'll store the prices in the `c` array and the coefficients in the utility function in the `α` variable.
* The $\alpha_{i}$ coefficients (because we have a linear utility function) are the [marginal utilities](https://en.wikipedia.org/wiki/Marginal_utility), i.e., they tell us the satisfaction we gain from consuming an additional unit of good $i$. They have units of `utils/qty`
* The $c_{i}$ coefficients represent the unit cost of each good, e.g., the cost of a single apple or orange. The $c_{i}$ coefficients have units of `USD/qty.`
* Finally, the `total_budget` variable holds the amount of money we spend on apples and oranges. The `total_budget` has units of `USD`.

In [2]:
# u = [0.55, 0.45]; # coefficients for case A (apples only)
u = [0.15, 0.55]; # coefficients for case B (oranges only)
# u = [2.0, 4.0]; # coefficients for case C (both)
c = [2.0 4.0]; # unit price of an Apple and an Orange
total_budget = 100.0; # total budget that we can spend

___

## Task 1: Compute the primal solution to the apple versus orange problem
In this task, we solve the `primal` linear programming problem for the unknown values in our problem, i.e., the number of apples or oranges we should purchase to maximize our happiness function. The problem we are solving is a linear programming problem of the form:

> __Fruit choice problem (primal)__
> 
> Let's choose between two fruits, `apples` and `oranges`. We have a fixed budget to spend on these fruits. The price of an apple and the price of an orange are specified above. We want to maximize our happiness (utility) from consuming these fruits, given our budget constraint.
> 
> $$
\begin{align*}
\text{maximize}~\mathcal{O}(\mathbf{x}) &= U\left(x_{1},\dots,x_{n}\right) \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}\;{x}_{i} & = I\\
\text{and}~x_{i}&\geq{0}\qquad{i=1,2,\dots,n}
\end{align*}
> $$
> 
> The $c_{i}\geq{0}~\forall{i}$ denotes the cost of object $i$, $x_{i}\geq{0}$ represents 
the amount of object $i$ purchased or consumed by the agent, and $I$ represents the budget that we have to spend. In this case, we'll use a __linear__ utility function of the form:
> $$
U(x) = u_{1}\cdot{x}_{1}+u_{2}\cdot{x}_{2}
> $$
> where $u_{1}$ are the marginal utilities (units: `utils/qty`), $x_{1}$ denotes the quantity of `apples = 1`, and $x_{2}$ represents the number of `oranges = 2` that we purchase.

To solve this problem, let's first specify the bounds of the variables in the bounds variable. The first column of the array corresponds to the lower bound, while the second column corresponds to the upper bound for the unknown variable(s) $x_{i}$.

> __Minimum and maximum values for the unknowns__
> 
> The minimum value for the unknowns $x_{i}$ is zero, i.e., we cannot purchase a negative amount of fruit. It would be tempting to set the upper bound to infinity; however, let's think about that choice for a moment. If we have an infinite budget and an unlimited supply, we could purchase an infinite amount of fruit. However, we have a limited budget. The maximum amount we could buy would be to spend our entire budget on one fruit. Thus, the upper bound for each unknown $x_{i}$ is $x_{j} = I/c_{i}$. 

Let's specify the bounds for our problem in the `primal_bounds::Array{Float64,2}` variable.

In [3]:
primal_bounds = let 
    
    # initialize -
    number_of_unknowns = 2; # we have two fruits to choose from
    bounds = zeros(number_of_unknowns, 2); # initialize the bounds
    I = total_budget; # shorthand for the budget

    # set the bounds for each unknown
    for i ∈ 1:number_of_unknowns
        bounds[i, 1] = 0.0; # lower bound is 0
        bounds[i, 2] = I / c[i]; # upper bound is I/c[i]
    end

    bounds; # return
end;

Next, we create an instance of [the `MyLinearProgrammingProblemModel` model](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MyLinearProgrammingProblemModel) using [a `build(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/factory/#VLDataScienceMachineLearningPackage.build) and store this model in the `primal_problem` variable. 
This model holds the data associated with the problem, e.g., the unit costs, the marginal utilities, the right-hand side vector (i.e., the budget), and the problem bounds.

In [4]:
primal_problem = build(MyLinearProgrammingProblemModel, (
    
    c = u, # coefficients in the Utility function (objective)
    A = c, # unit prices of x1 and x2 (we need this as a matrix in this formulation)
    b = [total_budget], # budget is the right-hand side
    
    # how much of x₁ and x₂ can we buy?
    lb = primal_bounds[:,1], # lower bound
    ub = primal_bounds[:,2] # upper bound
));

Finally, we pass the `primal_problem` variable to [the `solve(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/solvers/#VLDataScienceMachineLearningPackage.solve), which constructs the linear program using the [JuMP domain-specific language](https://jump.dev/). 

The implementation of [the `solve(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/solvers/#VLDataScienceMachineLearningPackage.solve) takes the data from the `primal_problem` instance, builds the various problem structures, and returns the solution in the `primal_solution` dictionary.

> __Why the try-catch block?__ The [solve(...) method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/solvers/#VLDataScienceMachineLearningPackage.solve) uses the [GLPK solver](https://en.wikipedia.org/wiki/GLPK) to solve the linear program. If the solver fails, the `solve(...)` method will throw an error. To prevent the notebook from crashing, we wrap the call to [the `solve(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/solvers/#VLDataScienceMachineLearningPackage.solve) in a `try-catch` block. If the solver fails, we catch the error and print a message to the user.

Solve the primal problem:

In [5]:
primal_solution_dictionary = let

    # initialize -
    primal_solution = nothing;
    try
        primal_solution = solve(primal_problem) # call the solver
    catch error
        println(error)
    end
    
    primal_solution; # return
end;

What's in the solution dictionary?

In [6]:
primal_solution_dictionary

Dict{String, Any} with 3 entries:
  "argmax"          => [0.0, 25.0]
  "status"          => OPTIMAL
  "objective_value" => 13.75

The `argmax` key points to the optimal values of the unknowns, i.e., the number of apples and oranges we should purchase to maximize our happiness function. The `objective_value` key points to the optimal value of the objective function, i.e., the maximum happiness we can achieve given our budget constraint. The `status` key holds the status of the optimization problem, e.g., whether it was solved successfully or not.

> __Note__: In the implementation of the `solve(...)` method we have an @assert statement that checks that the status of the optimization problem is optimal. If the status is not optimal, the @assert statement will throw an error. This is a safety check to ensure that we only proceed with the solution if the optimization problem was solved successfully.

So, which fruit should we buy (and how much of each)? It depends on the coefficients in the utility function (the marginal utilities). Try changing the values in the `u` variable and re-running the notebook to see how the solution changes.

In [7]:
let

    # initialize -
    df = DataFrame();
    labels = ["apples", "oranges"];
    x_opt = primal_solution_dictionary["argmax"]; # get the optimal values of the unknowns
    number_of_unknowns = length(x_opt); # number of unknowns

    # populate the dataframe -
    for i ∈ 1:number_of_unknowns
        row_df = (
            fruit = labels[i],
            quantity = x_opt[i]
        )
        push!(df, row_df)
    end

    pretty_table(df, backend = :text,
         table_format = TextTableFormat(borders = text_table_borders__compact))
end

 --------- ----------
    fruit   quantity 
   String    Float64 
 --------- ----------
   apples        0.0
  oranges       25.0
 --------- ----------


___

## Task 2: Compute the dual solution to the apple versus orange problem
In this task, we solve the `dual` linear programming problem for the unknown values in our problem, i.e., the shadow prices associated with the budget constraint. The _dual problem_ has the form:
$$
\begin{aligned}
\text{minimize}\quad & \sum_{j=1}^{m} b_{j}\,y_{j}\\
\text{subject to}\quad & \sum_{j=1}^{m} A_{i,j}\,y_{j}\;\ge\;c_{i}
\quad&&i=1,2,\dots,n,\\
&y_{j}\;\ge\;0
\quad&&j=1,2,\dots,m.
\end{aligned}
$$

### What has changed?
There are several key differences between the primal and dual linear programming problems:
1. The objective function flips (maximum ⇒ minimum or minimum ⇒ maximum).
2. Primal objective coefficients $c_i$ become the dual right-hand side constants.
3. Primal right-hand side constants $b_j$ become the dual objective coefficients.
4. The $m\times n$ constraint matrix $A$ is transposed in the dual (so $A^\top$ appears).
5. The number of variables and constraints swap: the primal has $n$ variables, $m$ constraints, and the dual has $m$ variables and $n$ constraints.
6. Each primal constraint $a_j^\top x \le b_j$ gives a dual variable $y_j$. Each primal variable $x_i$ gives a dual constraint $(A^\top y)_i \ge c_i$.
7. Inequality directions and sign restrictions invert for the constraints: A $\le$ constraint in the primal gives rise to a $\ge$ constraint in the dual (and vice versa).
8. Equality constraints in the primal become free variables in the dual, i.e., $a_j^T x = b_j$ gives rise to a dual variable $y_j$ that is free (no sign restriction), while a dual constraint $A^\top y \ge c$ gives rise to a primal variable $x_i$ that is free.

Finally, the solutions of the primal and dual problems are related by the concept of __duality__. For a primal problem: $\max\{\,c^T x : A x \le b,\;x\ge0\}$ and its corresponding dual problem: $\min\{\,b^T y : A^T y \ge c,\;y\ge0\}$, the solutions are related:
* __Weak duality__: For any primal feasible $x$ and dual feasible $y$, we have $c^T x \le b^T y$. Thus, the primal optimum is always bounded above by the dual optimum. The difference between the two is called the _duality gap_.
* __Strong duality__: If both primal and dual are feasible and have finite optimal values, then $\max\{\,c^T x \} = \min\{\,b^T y\}$, i.e., the _duality gap is zero_. This means that the optimal values of the primal and dual problems are equal.

Let's set up and solve the dual problem. First, we need to set the bounds for the dual variables. In this case, the dual variables represent the shadow prices associated with the budget constraint. The shadow prices must be non-negative, so we set the lower bound to zero and the upper bound to infinity.

In [8]:
dual_bounds = let 
    
    # initialize -
    number_of_dual_variables = 1; # we have one constraint (the budget constraint)
    bounds = zeros(number_of_dual_variables, 2); # initialize the bounds

    # set the bounds for each dual variable
    for j ∈ 1:number_of_dual_variables
        bounds[j, 1] = 0.0; # lower bound is 0
        bounds[j, 2] = Inf; # upper bound is infinity
    end

    bounds; # return
end;

Next, we create an instance of [the `MyLinearProgrammingProblemModel` model](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MyLinearProgrammingProblemModel) using [a `build(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/factory/#VLDataScienceMachineLearningPackage.build) and store this model in the `dual_problem` variable.

In [9]:
dual_problem = build(MyLinearProgrammingProblemModel, (
    
    c = [-I], # coefficients in the objective function (negative because we are minimizing)
    A = c, # unit prices of x1 and x2 (we need this as a matrix in this formulation)
    b = -u, # the rigfht-hand side (Ay >= c) so we need to negate u = c
    
    # how much of x₁ and x₂ can we buy?
    lb = dual_bounds[:,1], # lower bound
    ub = dual_bounds[:,2] # upper bound
));

MethodError: MethodError: no method matching VecOrMat{Float64}(::Vector{UniformScaling{Int64}})
The type `VecOrMat{Float64}` exists, but no method is defined for this combination of argument types when trying to construct it.

## Key Takeaways

> __What did we learn from this example?__
>
> * **Linear optimization with budget constraints produces corner solutions** - you'll typically buy all of one item or all of another, with the choice determined by which fruit gives higher marginal utility per dollar spent.
> * **Economic intuition aligns with mathematical results** - the slopes of budget lines and indifference curves predict the solution, and the special case of equal ratios leads to multiple optimal solutions.
> * **Julia's optimization ecosystem makes implementation straightforward** - proper problem setup with bounds, error handling, and solution interpretation are the key skills, not the computational mechanics.

Try changing the `u` coefficients in the second code cell and re-running the notebook to see these principles in action!
___